In [33]:
import tushare as ts
import pandas as pd
import datetime

In [34]:
#basics = ts.get_stock_basics()
#basics.to_csv('e:/quant/data/bascis.json')
basics = pd.read_csv('e:/quant/data/bascis.json')
data = basics.iloc[0]
print data
timeToMarket = data['timeToMarket']
print timeToMarket
startDate = datetime.datetime(timeToMarket/10000 ,timeToMarket%10000/100 , timeToMarket%10000%100) 
print startDate.strftime('%Y-%m-%d')
#sh=ts.get_h_data('000001' , start=timeToMarket[0:4]+'-'+timeToMarket[4:6]+'-'+timeToMarket[6:] , end='2016-02-18') 
#print sh.head()




code                          2
name                      万  科Ａ
industry                   全国地产
area                         深圳
pe                        29.51
outstanding              970833
totals              1.10391e+06
totalAssets         5.70793e+07
liquidAssets        5.20001e+07
fixedAssets              404016
reserved                 834237
reservedPerShare           0.76
esp                       0.621
bvps                       8.05
pb                         3.04
timeToMarket           19910129
Name: 1, dtype: object
19910129
1991-01-29
